# Exploring `build_bus_regions` outputs

### Jupyter notebook needs to be improved

In [ ]:
import xarray as xr
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import pypsa
import shutil
from rasterio.plot import show

import os
import sys
sys.path.append('../')  # to import helpers
from scripts._helpers import sets_path_to_root
sets_path_to_root("pypsa-africa")

# Input

In [2]:
# Load network
network_path = os.getcwd() + "/networks/base.nc"
n = pypsa.Network(network_path)
n

In [3]:
buses = n.buses
# Adding geometries from x and y coordinates
buses["geometry"] = gpd.points_from_xy(buses.lon, buses.lat)
buses = gpd.GeoDataFrame(buses, crs='epsg:4326')

continent = gpd.read_file('resources/country_shapes.geojson')
off_shore = gpd.read_file('resources/offshore_shapes.geojson')

ax = continent.plot(figsize=(10,10))
off_shore.plot(ax=ax)
buses.plot(ax=ax, color = "orange", markersize = 1)

In [5]:
buses[buses.country=="NG"]

# Output

In [6]:
continent = gpd.read_file('resources/regions_onshore.geojson')
off_shore = gpd.read_file('resources/regions_offshore.geojson')

ax = continent.plot(figsize=(10,10))
off_shore.plot(ax=ax, color="red")

In [8]:
c_b = n.buses.country == "NG"
n.buses.substation_off
b = (n.buses.substation_off & c_b)
b.sum()

In [9]:
c_b.sum()

In [10]:
n.buses.groupby("country").sum()

In [12]:
from shapely.geometry import LineString, Point

lines = n.lines[n.lines.country == "NG"].copy()
# lines = n.lines.copy()
lines["geometry"] = [LineString([Point(x.bus0_lon, x.bus0_lat), Point(x.bus1_lon, x.bus1_lat)]) for index, x in lines.iterrows() 
    if (x.bus0_lat is not None) and (x.bus0_lon is not None) and (x.bus1_lat is not None) and (x.bus1_lon is not None)]
lines = gpd.GeoDataFrame(geometry=lines.geometry, crs='epsg:4326')

continent = gpd.read_file('resources/country_shapes.geojson')
off_shore = gpd.read_file('resources/offshore_shapes.geojson')

ax = continent.plot(figsize=(10,10))
off_shore.plot(ax=ax)
c_code = "CD"
# lines[lines.country==c_code].plot(ax=ax, color="yellow")
# buses[buses.country==c_code].plot(ax=ax, color = "orange", markersize = 1)
lines.plot(ax=ax, color="yellow")
buses.plot(ax=ax, color = "orange", markersize = 1)

In [69]:
lines.country.unique()